## Implementation

In [1]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

In [2]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()

input = torch.tensor(np.array(data[['Open']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close']), dtype=torch.float32)

# Example 1

## Set up a simple model manully

In [8]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=100, tol=1e-3)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[ 0.2200,  0.4409],
         [ 1.9199,  0.9252]],

        [[-0.4395,  0.0823],
         [-0.4329, -1.1235]]], requires_grad=True)
Parameter containing:
tensor([[ 0.2997, -0.1962],
        [-0.1452,  0.2960]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [11]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[0.4355, 0.5645],
        [0.3918, 0.6082],
        [0.4437, 0.5563],
        [0.4290, 0.5710],
        [0.4636, 0.5364],
        [0.4463, 0.5537],
        [0.4562, 0.5438],
        [0.4460, 0.5540],
        [0.4531, 0.5469],
        [0.4472, 0.5528]])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [12]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[0.4570, 0.5430],
        [0.4091, 0.5909],
        [0.4136, 0.5864],
        [0.4296, 0.5704],
        [0.4429, 0.5571],
        [0.4570, 0.5430],
        [0.4480, 0.5520],
        [0.4583, 0.5417],
        [0.4387, 0.5613],
        [0.4663, 0.5337]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [13]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[0.3937, 0.6063],
        [0.3084, 0.6916],
        [0.3599, 0.6401],
        [0.3613, 0.6387],
        [0.4073, 0.5927],
        [0.4042, 0.5958],
        [0.4050, 0.5950],
        [0.4052, 0.5948],
        [0.3930, 0.6070],
        [0.4140, 0.5860]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [14]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[0.0368, 0.4143],
         [0.4414, 0.1076]],

        [[0.0265, 0.4149],
         [0.4826, 0.0759]],

        [[0.0250, 0.4535],
         [0.4347, 0.0868]],

        [[0.0306, 0.4144],
         [0.4677, 0.0872]],

        [[0.0333, 0.4221],
         [0.4375, 0.1071]],

        [[0.0409, 0.3841],
         [0.4520, 0.1229]],

        [[0.0390, 0.3873],
         [0.4425, 0.1313]],

        [[0.0391, 0.3990],
         [0.4463, 0.1155]],

        [[0.0362, 0.3916],
         [0.4530, 0.1191]],

        [[0.0398, 0.4074],
         [0.4368, 0.1160]]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [15]:
IOHMM._baum_welch()

0
Iteration:  0 
Initial pi:
 Parameter containing:
tensor([0.6073, 0.6104], requires_grad=True) 
Transition matrix:
 Parameter containing:
tensor([[[ 0.2200,  0.4409],
         [ 1.9199,  0.9252]],

        [[-0.4395,  0.0823],
         [-0.4329, -1.1235]]], requires_grad=True) 
Emission matrix:
 Parameter containing:
tensor([[ 0.2997, -0.1962],
        [-0.1452,  0.2960]], requires_grad=True) 
Standard deviation:
 Parameter containing:
tensor([1., 1.], requires_grad=True)
1
Iteration:  1 
Initial pi:
 Parameter containing:
tensor([0.7372, 0.7454], requires_grad=True) 
Transition matrix:
 Parameter containing:
tensor([[[ 0.2200,  0.4409],
         [ 1.9199,  0.9252]],

        [[-0.4395,  0.0823],
         [-0.4329, -1.1235]]], requires_grad=True) 
Emission matrix:
 Parameter containing:
tensor([[ 0.2997, -0.1962],
        [-0.1452,  0.2960]], requires_grad=True) 
Standard deviation:
 Parameter containing:
tensor([1., 1.], requires_grad=True)
2
Iteration:  2 
Initial pi:
 Parameter co

## See the training results

In [33]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([9640185.0000, 2007872.8750], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[-0.1955, -0.9656],
         [ 0.4224,  0.2673]],

        [[-0.4212, -0.5107],
         [-1.5727, -0.1232]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[ 3.5870, -1.8313],
        [ 1.5987, -1.2770]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)


## Viterbi

In [34]:
#call the viterbi algorithm
IOHMM.viterbi()